In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from semantic_text_splitter import TextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
# import faiss
import os

# SET YOU ENVIRONMENT API KEY HERE

In [25]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"}
)

vectordb = FAISS.load_local(
    folder_path="./faiss_db",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

retriever1 = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retriever2 = vectordb.as_retriever(search_type = "mmr", search_kwargs={"k": 5})

In [27]:
query = input("Write your query: ")
retrived_docs_1 =retriever1.invoke(query)
retrived_docs_2 =retriever2.invoke(query)
# print(retrived_docs[0].page_content)

In [28]:
def full_context(docs1, docs2):
    all_docs = docs1 +docs2
    context_text = "\n\n".join(doc.page_content for doc in all_docs)
    return context_text

In [29]:
template = PromptTemplate(template='''you are an helpful research assistant
                          give answer to the given query: {query} .
                          answer only from the given context: {context}
                          if the context is insufficient answer say that the knowledge base is insufficient''',
                          input_variables= ["query", "context"])

In [30]:
prompt = template.invoke({"query" : query, "context"  : full_context(retrived_docs_1, retrived_docs_2)})

In [31]:
answer = model.invoke(prompt)
print(answer.content)

 Based on the provided context, the statistical definition of temperature is derived in Chapter 4 of the textbook. The temperature of a system is defined from a statistical perspective using the Boltzmann distribution. This distribution describes how a thermal system behaves when it is placed in thermal contact with a large thermal reservoir. The concept of temperature is related to the average energy of motion of particles in a system, and it is determined by the laws of probability. The text mentions that the temperature is proportional to the grand thermodynamic absolute temperature, which is also related to the idea of a Carnot engine. However, the context does not provide a detailed derivation of the statistical definition of temperature from first principles. It is recommended to refer to the textbook for a more comprehensive understanding of this topic.
